In [1]:
import numpy as np
from os.path import normpath as fn # Fix Linux/Windows path issue
import sys
sys.path.append("nn") # add the nn module into system path

import nn.layer as layer
import nn.graph as graph
import nn.solver as solver
import nn.container as container
import nn.loss as loss

np.random.seed(0)

In [2]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))

train_im = np.float32(data['im_train'])/255.-0.5
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_lb = data['lbl_val']

In [3]:
###################################
# build static computational graph
###################################
nHidden = 2048

# placeholder for input
inp = layer.Tensor()
lab = layer.Tensor()

model = container.Sequential([layer.Linear(28*28, nHidden),
                           layer.RELU(),
                        #    layers.Dropout(0.3),
                           layer.Linear(nHidden, 10)
                          ])
print(model)

# loss function: softmax + crossentropy
smaxloss = loss.SmaxCELoss()
accuracy = loss.Accuracy()

# build computational graph
y = model(inp)

# Cross Entropy of Soft-max
criterion = smaxloss(y,lab)

# Accuracy
acc = accuracy(y,lab)

(main) Sequential(
  (0) Linear(in_features=784, out_features=2048)
  (1) RELU()
  (2) Linear(in_features=2048, out_features=10)
)


In [4]:
lr=0.001
NUM_EPOCH=10
DISPITER=100
BSZ=64

optimizer = solver.Momentum(graph.params, lr, mom=0.9)

In [5]:
# Training loop
batches = range(0,len(train_lb)-BSZ+1,BSZ)
niter, avg_loss, avg_acc=0, 0., 0.

for ep in range(NUM_EPOCH+1):
    # As we train, let's keep track of val accuracy
    vacc, vloss, viter= 0., 0., 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        inp.set(val_im[b:b+BSZ,:])
        lab.set(val_lb[b:b+BSZ])
        graph.Forward()
        viter += 1
        vacc += acc.top
        vloss += criterion.top
    vloss, vacc = vloss / viter, vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break

    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],:])
        lab.set(train_lb[idx[b:b+BSZ]])

        graph.Forward()
        avg_loss += criterion.top 
        avg_acc += acc.top
        niter += 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER
            avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss, avg_acc = 0., 0.

        graph.Backward(criterion)
        optimizer.step()

000000000: #### 0 Epochs: Val Loss = 2.316e+00, Accuracy = 12.92%
000000100: Training Loss = 1.769e+00, Accuracy = 55.66%
000000200: Training Loss = 1.061e+00, Accuracy = 79.69%
000000300: Training Loss = 7.699e-01, Accuracy = 84.88%
000000390: #### 1 Epochs: Val Loss = 5.632e-01, Accuracy = 88.12%
000000400: Training Loss = 6.561e-01, Accuracy = 84.41%
000000500: Training Loss = 5.653e-01, Accuracy = 86.81%


KeyboardInterrupt: 